---
title: "Einführung in Maschinelle Lernverfahren"
jupyter: ir
---


In [ ]:
#| include: false
req_pkg <- c(
  "riskCommunicator", "data.table", "performance", "tinyplot", "see",
  "splitTools", "kdry", "kableExtra", "tidymodels", "kernlab", "parsnip",
  "yardstick", "broom", "rpart.plot", "ranger", "DALEXtra", "shapviz",
  "fastshap", "vip"
)
for (r in req_pkg) {
  if (!(r %in% installed.packages()[, "Package"])) {
    install.packages(r)
  }
}


In [ ]:
#| echo: true
dataset_full <- riskCommunicator::framingham |>
  data.table::data.table() # Daten einlesen
# Subset: Basisuntersuchung
dataset <- dataset_full[get("PERIOD") == 1, ]

# Relevante Spalten definieren
use_cols <- c("SEX", "TOTCHOL", "AGE", "SYSBP",
"CURSMOKE", "CIGPDAY", "BMI", "DIABETES",
"HYPERTEN")
# Relevante Spalten filtern, fehlende Werte entfernen
dataset <- dataset[
  , .SD, .SDcols = use_cols
] |>na.omit()

# Transformieren der katgeorialen Variablen
# "SEX" "CURSMOKE" "CIGPDAY" "DIABETES" "HYPERTEN"
cat_vars <- use_cols[c(1, 5, 8, 9)]
# Datentyp "factor" ändern
dataset[, (cat_vars) := lapply(
  X = .SD,
  FUN = factor),
  .SDcols = cat_vars
]


In [ ]:
# Übersicht über den Datensatz --> n=4332 Beobachtungen
str(dataset)


In [ ]:
#| echo: true
# Ausgabe der ersten 10 Zeilen
dataset[1:10, ]


In [ ]:
#| echo: true
# Die Funktion erzeugt eine Liste mit Indices für die jeweiligen Datensets
# Das stratifizierte Splitten anhand der Zielvariable "HYPERTEN" soll deren
# gleichmäßige Verteilung in den Teildatensätzen sicherstellen.
data_splits <- splitTools::partition(
  y = dataset$HYPERTEN,
  p = c(train = 0.7, validation = 0.15, test = 0.15),
  type = "stratified",
  seed = 123
)
sapply(data_splits, length)  # Verteilung der Beobachtungen auf die Teildatensätze


In [ ]:
#| include: false
# Initialisierung einer leeren Tabelle und Sammeln der kontinuierlichen / diskreten Variablen in je einem Vektor
results_table <- data.table::data.table()
num_vars <- dataset[, sapply(.SD, is.numeric), .SDcols = colnames(dataset)]
# Diskrete Variablen ohne 'CIGPDAY' (zu viele Ausprägungen für Visualisierung)
cat_vars <- dataset[, sapply(.SD, is.factor), .SDcols = setdiff(colnames(dataset), "CIGPDAY")]

# Helper-Funktion definieren für Counts der diskreten Variablen
level_counts <- function(X, cols) {
  lvls <- levels(cols[, get(X)])
  out <- sapply(
    X = lvls,
    FUN = function(x) {
      kdry::rep_frac_pct(
        count = cols[get(X) == x, .N],
        count_reference = cols[, .N]
      )
    },
    simplify = TRUE
  )
  names(out) <- paste(X, names(out), sep = "=")
  out <- out |>
    cbind() |>
    data.table::data.table(keep.rownames = TRUE)
  return(out)
}

# Berechnung von Mittelwert/Standardabweichung (kontinuierlichen Variablen) bzw.
# Summe/relative Häufigkeit (diskrete Variablen) für Teildatensätze
for (sp in names(data_splits)) {
  num.cols <- dataset[data_splits[[sp]], names(num_vars)[num_vars], with = FALSE]
  cat.cols <- dataset[data_splits[[sp]], names(cat_vars)[cat_vars], with = FALSE]

  add_col_num <- sapply(X = num.cols, FUN = kdry::rep_mean_sd) |>
    cbind() |>
    data.table::data.table(keep.rownames = TRUE)
  colnames(add_col_num)[2] <- "out"

  add_col_cat <- lapply(X = names(cat.cols), FUN = level_counts, cols = cat.cols) |>
    data.table::rbindlist()

  results_table <- cbind(results_table, rbind(add_col_num, add_col_cat))
}

# Redundate Spalten entfernen
results_table <- results_table[, c(1, 2, 4, 6)];

# Spaltenbeschriftungen
colnames(results_table) <- c("Variable", names(data_splits))
# jeweiliges N an die Split-Bezeichnungen in den Spaltennamen ergänzen
split_colnames <- sapply(X = names(data_splits), FUN = function(x) {paste0(x, " (n=", length(data_splits[[x]]), ")")})
colnames(results_table)[2:4] <- split_colnames


In [ ]:
#| echo: true
knitr::kable(results_table, caption = "Train-Validation-Test Split") |>
  kableExtra::kable_styling(font_size = "70%")


In [ ]:
#| echo: true
# Teildatensätze für Regressions- und Klassifizierungs-Beispiele

# Regression: Zielvariable "SYSBP" --> Entfernen von "HYPERTEN"
dataset_reg <- dataset[
  , .SD, .SDcols = setdiff(colnames(dataset), "HYPERTEN")
]

# Klassifikation: Zielvariable "HYPERTEN" --> Entfernen von "SYSBP"
dataset_cls <- dataset[
  , .SD, .SDcols = setdiff(colnames(dataset), "SYSBP")
]


In [ ]:
#| echo: true
#| out-width: 100%
#| fig-align: center
boxplot(dataset$SYSBP ~ dataset$HYPERTEN)


In [ ]:
#| echo: true
set.seed(123)
# Definition des SVM-Modells
svm_spec <- parsnip::svm_rbf() |>
  parsnip::set_mode("classification") |>
  parsnip::set_engine("kernlab", scaled = TRUE)

# Fitten des Modells auf die Daten
# "cost" entspricht dem Tuning-Parameter 'C'
svm_m1 <- svm_spec |>
  parsnip::set_args(cost = 500) |>
  parsnip::fit(HYPERTEN ~ .,
  data = dataset_cls[data_splits$train, ])

# Vorhersagen werden auf dem Test-Datensatz berechnet
svm_m1_preds <- broom::augment(
  x = svm_m1, new_data = dataset_cls[data_splits$test, ]
)


In [ ]:
#| echo: true
svm_m1


In [ ]:
#| echo: true
#| out-width: 60%
#| fig-align: center
yardstick::roc_curve( # ROC-Curve
  data = svm_m1_preds, truth = HYPERTEN,
  .pred_1, event_level = "second"
) |> ggplot2::autoplot()


In [ ]:
#| echo: true
# Accuracy und AU-ROC
auc <- svm_m1_preds |>
  yardstick::accuracy(
    truth = HYPERTEN, estimate = .pred_class
  )
roc <- svm_m1_preds |>
  yardstick::roc_auc(
    truth = HYPERTEN, .pred_1,
    event_level = "second"
  )
rbind(auc, roc)


In [ ]:
#| echo: true
set.seed(123)
# Definition des SVM-Modells
svm_spec <- parsnip::svm_rbf() |>
  parsnip::set_mode("regression") |>
  parsnip::set_engine("kernlab", scaled = TRUE)

# Fitten des Modells auf die Daten
# "cost" entspricht dem Tuning-Parameter 'C'
svm_m2 <- svm_spec |>
  parsnip::set_args(cost = 500) |>
  parsnip::fit(SYSBP ~ ., data = dataset_reg[data_splits$train, ])

# Vorhersagen werden auf dem Test-Datensatz berechnet
svm_m2_preds <- broom::augment(
  x = svm_m2, new_data = dataset_reg[data_splits$test, ]
)


In [ ]:
#| echo: true
svm_m2

# RMSE
yardstick::rmse(
  data = svm_m2_preds,
  truth = SYSBP,
  estimate = .pred
)


In [ ]:
#| echo: true
set.seed(123)
# Definition des Regression Trees
tree_spec <- parsnip::decision_tree() |>
  parsnip::set_engine("rpart")
reg_tree_spec <- tree_spec |>
  parsnip::set_mode("regression")

# Fitten des Modells auf die Daten
# Zielvariable: 'SYSBP' aufgrund des Regressions-Settings
tree_m1 <- reg_tree_spec |>
  parsnip::set_args(model = TRUE) |>
  parsnip::fit(
    SYSBP ~ ., data = dataset_reg[data_splits$train, ]
  )

# Vorhersagen werden auf dem Test-Datensatz berechnet
tree_m1_preds <- broom::augment(
  x = tree_m1, new_data = dataset_reg[data_splits$test, ]
)


In [ ]:
#| echo: true
tree_m1


In [ ]:
#| echo: true
#| out-width: 90%
#| fig-align: center
tree_m1 |>
  parsnip::extract_fit_engine() |>
  rpart.plot::rpart.plot()


In [ ]:
#| echo: true
# RMSE
yardstick::rmse(
  data = tree_m1_preds,
  truth = SYSBP,
  estimate = .pred
)


In [ ]:
#| echo: true
set.seed(123)
# Definition des Classification Trees
cls_tree_spec <- tree_spec |>
  parsnip::set_mode("classification")

cntrl <- rpart::rpart.control(
  cp = 0.0075, # Schwellenwert für Informationszuwachs
  minbucket = 20 # Mindestanzahl an Beobachtungen im Endknoten
)

# Fitten des Modells auf die Daten
# Zielvariable: 'HYPERTEN' aufgrund des Klassifikations-Settings
tree_m2 <- cls_tree_spec |>
  parsnip::set_args(model = TRUE, control = cntrl) |>
  parsnip::fit(
    HYPERTEN ~ ., data = dataset_cls[data_splits$train, ]
  )

# Vorhersagen werden auf dem Test-Datensatz berechnet
tree_m2_preds <- broom::augment(
  x = tree_m2, new_data = dataset_cls[data_splits$test, ]
)

# Accuracy und AU-ROC
auc <- tree_m2_preds |>
  yardstick::accuracy(truth = HYPERTEN, estimate = .pred_class)
roc <- tree_m2_preds |>
  yardstick::roc_auc(truth = HYPERTEN, .pred_1, event_level = "second")


In [ ]:
#| echo: true
tree_m2
rbind(auc, roc)


In [ ]:
#| echo: true
#| out-width: 100%
#| fig-align: center
tree_m2 |>
  parsnip::extract_fit_engine() |>
  rpart.plot::rpart.plot()


In [ ]:
#| echo: true
#| out-width: 60%
#| fig-align: center
yardstick::roc_curve( # ROC-Curve
  data = tree_m2_preds, truth = HYPERTEN, .pred_1, event_level = "second"
) |> ggplot2::autoplot()


In [ ]:
#| echo: true
set.seed(123)
# Definition des Random Forests
rf_spec <- parsnip::rand_forest(
  trees = 1000, # B
  mtry = floor(sqrt(ncol(dataset_cls))) # m
) |> parsnip::set_engine("ranger")
cls_rf_spec <- rf_spec |>
  parsnip::set_args(replace = FALSE) |>
  parsnip::set_mode("classification")

# Fitten des Modells auf die Daten
# Zielvariable: 'HYPERTEN' aufgrund des Klassifikations-Settings
rf_m1 <- cls_rf_spec |>
  parsnip::fit(HYPERTEN ~ ., data = dataset_cls[data_splits$train, ])

# Vorhersagen werden auf dem Test-Datensatz berechnet
rf_m1_preds <- broom::augment(x = rf_m1, new_data = dataset_cls[data_splits$test, ])

# Accuracy und AU-ROC
auc <- rf_m1_preds |> yardstick::accuracy(truth = HYPERTEN, estimate = .pred_class)
roc <- rf_m1_preds |> yardstick::roc_auc(truth = HYPERTEN, .pred_1, event_level = "second")


In [ ]:
#| echo: true
rf_m1
rbind(auc, roc)


In [ ]:
#| echo: true
#| out-width: 60%
#| fig-align: center
yardstick::roc_curve( # ROC-Curve
  data = rf_m1_preds,
  truth = HYPERTEN,
  .pred_1,
  event_level = "second"
) |> ggplot2::autoplot()


In [ ]:
#| echo: true
set.seed(123)
rf_spec <- parsnip::rand_forest(
  trees = 1000, # B
  mtry = floor(ncol(dataset_cls) / 3) # m
) |> parsnip::set_engine("ranger")
reg_rf_spec <- rf_spec |>
  parsnip::set_args(replace = FALSE) |>
  parsnip::set_mode("regression")

# Fitten des Modells auf die Daten
# Zielvariable: 'SYSBP' aufgrund des Regressions-Settings
rf_m2 <- reg_rf_spec |>
  parsnip::fit(
    SYSBP ~ ., data = dataset_reg[data_splits$train, ]
  )

# Vorhersagen werden auf dem Test-Datensatz berechnet
rf_m2_preds <- broom::augment(
  x = rf_m2, new_data = dataset_reg[data_splits$test, ]
)


In [ ]:
#| echo: true
# RMSE
yardstick::rmse(
  data = rf_m2_preds,
  truth = SYSBP,
  estimate = .pred
)


In [ ]:
#| echo: true
# Variable importance: "impurity"
reg_rf_spec_vip_imp <- rf_spec |>
  parsnip::set_args(replace = FALSE, importance = "impurity") |>
  parsnip::set_mode("regression")

rf_m2_vip_imp <- reg_rf_spec_vip_imp |>
  parsnip::fit( SYSBP ~ ., data = dataset_reg[data_splits$train, ])

# Variable importance: "permutation"
reg_rf_spec_vip_perm <- rf_spec |>
  parsnip::set_args(replace = FALSE, importance = "permutation") |>
  parsnip::set_mode("regression")

rf_m2_vip_perm <- reg_rf_spec_vip_perm |>
  parsnip::fit(SYSBP ~ ., data = dataset_reg[data_splits$train, ])


In [ ]:
#| echo: true
#| out-width: 100%
#| fig-align: center
vip::vip(rf_m2_vip_imp) +
  ggplot2::ggtitle(
    "Variable Importance: 'impurity'",
    subtitle = "Random Forest"
  )


In [ ]:
#| echo: true
#| out-width: 100%
#| fig-align: center
vip::vip(rf_m2_vip_perm) +
  ggplot2::ggtitle(
    "Variable Importance: 'permutation'",
    subtitle = "Random Forest"
  )


In [ ]:
#| echo: false
#| eval: false
# # Prediction wrapper
# pfun <- function(object, newdata) {
#   # muss einen numerischen Vektor zurückgeben
#   as.vector(kernlab::predict(object, newdata))
# }
# # Berechnung der Shapley-Werte
# s <- fastshap::explain(
#   object = parsnip::extract_fit_engine(svm_m2),
#   X = subset(dataset_reg[data_splits$train, ], select = -SYSBP),
#   newdata = subset(dataset_reg[data_splits$test, ], select = -SYSBP),
#   pred_wrapper = pfun, nsim = 1
# )
# # Erzeugen des Visualisierungs-Objects
# shap_freq <- shapviz::shapviz(object = s, X = dataset_reg[data_splits$test, ])


In [ ]:
#| echo: true
# Prediction wrapper
pfun <- function(object, newdata) {
  # muss einen numerischen Vektor zurückgeben
  as.vector(predict(object, newdata))$predictions
}
# Berechnung der Shapley-Werte für Random Forest
s <- fastshap::explain(
  object = parsnip::extract_fit_engine(rf_m2),
  X = subset(dataset_reg[data_splits$train, ], select = -SYSBP),
  newdata = subset(dataset_reg[data_splits$test, ], select = -SYSBP),
  pred_wrapper = pfun, nsim = 1
)
# Erzeugen des Visualisierungs-Objects
shap_freq <- shapviz::shapviz(object = s, X = dataset_reg[data_splits$test, ])


In [ ]:
#| echo: true
#| out-width: 80%
#| fig-align: center
shapviz::sv_importance(shap_freq)


In [ ]:
#| echo: true
#| out-width: 80%
#| fig-align: center
shapviz::sv_importance(shap_freq, kind = "bee")


In [ ]:
#| echo: true
#| out-width: 80%
#| fig-align: center
shapviz::sv_dependence(shap_freq, v = colnames(dataset_reg)[c(1:3, 5:7)])